In [1]:
from sage.matrix.constructor import matrix

def nullspace_of_polynomialsR(S):
    # Get the monomials appearing in all polynomials
    all_monomials = set()
    for poly in S:
        all_monomials.update(poly.monomials())
    
    # Create coordinate vectors for each polynomial
    coord_vectors = []
    for polynomial in S:
        coeffs = [polynomial.monomial_coefficient(monomial) for monomial in all_monomials]
        coord_vectors.append(vector(coeffs))
    
    # Matrix with coordinate vectors as columns
    A = matrix(coord_vectors).transpose()
    
    # Print original matrix
    print("Original Matrix:")
    print(A)

    # Rank of the matrix
    rank = A.rank()
    
    print("Matrix in Reduced Row Echelon form:")
    print(A.rref())
    
    # Nullspace of the matrix
    nullspace = A.right_kernel()
    
    # Check if polynomials are linearly independent
    lin_indp = 1 if rank == len(S) else 0
    
    return lin_indp, rank, nullspace

# Example usage
R.<x,y> = PolynomialRing(ZZ, ['x', 'y'], order='degrevlex')
f1 = x^2
f2 = x^1
f3 = x^3
f4 = 4*(x^5 + y)
S = {f1, f2, f3, f4}

S1 = {f1, f2, f3}

lin_indp, rank, nullspace = nullspace_of_polynomialsR(S)
print("Rank of the matrix:", rank, "Lin indp:",lin_indp)
print("Basis for the nullspace:")
print(nullspace.basis())

lin_indp, rank, nullspace = nullspace_of_polynomialsR(S1)
print("Rank of the matrix:", rank, "Lin indp:",lin_indp)
print("Basis for the nullspace:")
print(nullspace.basis())

Original Matrix:
[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 4]
[0 0 0 4]
Matrix in Reduced Row Echelon form:
[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]
[0 0 0 0]
Rank of the matrix: 4 Lin indp: 1
Basis for the nullspace:
[

]
Original Matrix:
[1 0 0]
[0 1 0]
[0 0 1]
Matrix in Reduced Row Echelon form:
[1 0 0]
[0 1 0]
[0 0 1]
Rank of the matrix: 3 Lin indp: 1
Basis for the nullspace:
[

]


In [3]:
from sage.matrix.constructor import matrix

def fglm(S, G1):
    # Initialize the new Groebner basis and the set of linearly independent polynomials
    G2 = list()
    independent_polynomials = list()
    independent_polynomials.append(R.gen(1)^0)
    check_polys = independent_polynomials.copy()
    
    original_polys = list()
    original_polys.append(R.gen(1)^0)
    # Iterate through each variable
    for i in [1, 0]:

        exp = 1
        while True:
            # Reduce the monomial by G1
            original = R.gen(i)^exp
            original_polys.append(original)
            print("original polys:",original_polys)
            
            reduced = (original).reduce(G1)
            print("Val:",R.gen(i),"Processing exponent:", exp, "Reduced:", reduced)


            print("i",i,"#1",independent_polynomials)

            check_polys.append(reduced)

            print("i",i,"#1",independent_polynomials)


            print("check_polys",check_polys,"independent_polynomials",independent_polynomials)
            # Check if the reduced monomial is a linear combination of elements in independent_polynomials
            lin_indp, _, nullspace = nullspace_of_polynomialsR(check_polys)
            print("reduced:",reduced,"linindp",lin_indp)
            if lin_indp:
                independent_polynomials.append(reduced)

                exp += 1 # inc exp
            else:
                # Extract coefficients from a vector in the nullspace

                basis_matrix = nullspace.basis_matrix()
                coefficients = basis_matrix[0]  # Take the first vector in the basis
                

                print("check_polys now:",check_polys,"original_polys now:",original_polys)

                print("coefficients",basis_matrix)
                print("current polys:",independent_polynomials)
                
                dot_product = coefficients * vector(original_polys)
                G2.append(dot_product)
                
                original_polys.pop()
                check_polys.pop()

                break

    return G2

# Example input
R.<x,y> = PolynomialRing(QQ,2,order='degrevlex')
f1 = y^3+x^2
f2 = x^2*y+x^2

S = [f1, f2]

I = ideal(f1, f2)*R

# Compute the Groebner basis G1 in degrevlex order
G1 = I.groebner_basis()

# New lex order Groebner basis
G2 = fglm(S, G1)

print("G1 (degrevlex order):", G1)
print("G2 (lex order):", G2)

original polys: [1, y]
Val: y Processing exponent: 1 Reduced: y
i 1 #1 [1]
i 1 #1 [1]
check_polys [1, y] independent_polynomials [1]
Original Matrix:
[1 0]
[0 1]
Matrix in Reduced Row Echelon form:
[1 0]
[0 1]
reduced: y linindp 1
original polys: [1, y, y^2]
Val: y Processing exponent: 2 Reduced: y^2
i 1 #1 [1, y]
i 1 #1 [1, y]
check_polys [1, y, y^2] independent_polynomials [1, y]
Original Matrix:
[1 0 0]
[0 1 0]
[0 0 1]
Matrix in Reduced Row Echelon form:
[1 0 0]
[0 1 0]
[0 0 1]
reduced: y^2 linindp 1
original polys: [1, y, y^2, y^3]
Val: y Processing exponent: 3 Reduced: -x^2
i 1 #1 [1, y, y^2]
i 1 #1 [1, y, y^2]
check_polys [1, y, y^2, -x^2] independent_polynomials [1, y, y^2]
Original Matrix:
[ 1  0  0  0]
[ 0  1  0  0]
[ 0  0  0 -1]
[ 0  0  1  0]
Matrix in Reduced Row Echelon form:
[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]
reduced: -x^2 linindp 1
original polys: [1, y, y^2, y^3, y^4]
Val: y Processing exponent: 4 Reduced: x^2
i 1 #1 [1, y, y^2, -x^2]
i 1 #1 [1, y, y^2, -x^2]
check_

In [4]:
from sage.matrix.constructor import matrix

def fglm(S, G1):
    # Initialize the new Groebner basis and the set of linearly independent polynomials
    G2 = list()
    independent_polynomials = list()
    independent_polynomials.append(R.gen(1)^0)
    check_polys = independent_polynomials.copy()
    
    original_polys = list()
    original_polys.append(R.gen(1)^0)
    # Iterate through each variable

    for i in [2, 1, 0]:  

        exp = 1
        while True:
            # Reduce the monomial by G1
            original = R.gen(i)^exp
            original_polys.append(original)
            print("original polys:",original_polys)
            
            reduced = (original).reduce(G1)
            print("Val:",R.gen(i),"Processing exponent:", exp, "Reduced:", reduced)

            print("i",i,"#1",independent_polynomials)

            check_polys.append(reduced)

            print("i",i,"#1",independent_polynomials)


            print("check_polys",check_polys,"independent_polynomials",independent_polynomials)
            # Check if the reduced monomial is a linear combination of elements in independent_polynomials
            lin_indp, _, nullspace = nullspace_of_polynomialsR(check_polys)
            print("reduced:",reduced,"linindp",lin_indp)
            if lin_indp:
                independent_polynomials.append(reduced)

                exp += 1 # inc exp
            else:
                # Extract coefficients from a vector in the nullspace

                basis_matrix = nullspace.basis_matrix()
                coefficients = basis_matrix[0]  # Take the first vector in the basis
                

                print("check_polys now:",check_polys,"original_polys now:",original_polys)

                print("coefficients",basis_matrix)
                print("current polys:",independent_polynomials)
                
                dot_product = coefficients * vector(original_polys)
                G2.append(dot_product)
                
                print("G2 is", G2)
                
                original_polys.pop()
                check_polys.pop()

                break
 
    return G2

R.<z,y,x> = PolynomialRing(QQ,3,order='degrevlex')
f1 = y^3 
f2 = x^2*y + x^2 + y + y^3
f3 = z*5 + y^3

S = [f1, f2, f3]

I = ideal(f1, f2, f3)*R

# Compute the Groebner basis G1 in degrevlex order
G1 = I.groebner_basis()
print("Original Groebner basis", G1)

# New lex order Groebner basis
G2 = fglm(S, G1)

print("G1 (degrevlex order):", G1)
print("G2 (lex order):", G2)

Original Groebner basis [x^4 - x^2 - y, y*x^2 + x^2 + y, y^2 - x^2 - y, z]
original polys: [1, x]
Val: x Processing exponent: 1 Reduced: x
i 2 #1 [1]
i 2 #1 [1]
check_polys [1, x] independent_polynomials [1]
Original Matrix:
[0 1]
[1 0]
Matrix in Reduced Row Echelon form:
[1 0]
[0 1]
reduced: x linindp 1
original polys: [1, x, x^2]
Val: x Processing exponent: 2 Reduced: x^2
i 2 #1 [1, x]
i 2 #1 [1, x]
check_polys [1, x, x^2] independent_polynomials [1, x]
Original Matrix:
[0 1 0]
[1 0 0]
[0 0 1]
Matrix in Reduced Row Echelon form:
[1 0 0]
[0 1 0]
[0 0 1]
reduced: x^2 linindp 1
original polys: [1, x, x^2, x^3]
Val: x Processing exponent: 3 Reduced: x^3
i 2 #1 [1, x, x^2]
i 2 #1 [1, x, x^2]
check_polys [1, x, x^2, x^3] independent_polynomials [1, x, x^2]
Original Matrix:
[0 1 0 0]
[1 0 0 0]
[0 0 0 1]
[0 0 1 0]
Matrix in Reduced Row Echelon form:
[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]
reduced: x^3 linindp 1
original polys: [1, x, x^2, x^3, x^4]
Val: x Processing exponent: 4 Reduced: x^2 